Finding the binding of TF to k-mers 

In [149]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from ortals_code.TF_review_in_seq import *
import ortals_code.TF_review_in_seq as ortals
from functools import reduce


In [65]:
# Useful CONSTS:
DATA_TF_PATH = "/home/labs/davidgo/ortalh/TF_analysis_data/data_TF_for_analysis"
PATH_SAVE_RESULTS = "/home/labs/davidgo/ortalh/TF_analysis_results/"
INPUTS_FOLDER = "/home/labs/davidgo/ortalh/TF_analysis_inputs/"
INPUT_CSV = "TF_analysis_inputs.csv"
TF_BINDING_THRESHOLD = 0.35
TF_BINDING_MINIMUM = 0.2
SLASH = "/"
COLORSMAPS = [plt.get_cmap('tab20b_r'), plt.get_cmap('gist_rainbow'), plt.get_cmap('tab20c_r'), plt.get_cmap('tab20_r')]
COLORS = []
K_NARROW = 20
HUMAN_GENOME_FILE = "hg19.fa"
OPP_NUC = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}

MPRA_DATA = "/home/labs/davidgo/Collaboration/humanMPRA/chondrocytes/comparative_analysis_combined/humanMPRA_with_seq_final2.csv"

COLUMN_NAMES = ["8-mer", "reverse_complement", "E-score", "Median", "Z-score"]

In [3]:
input_file_path = os.path.join(INPUTS_FOLDER, INPUT_CSV)
data_df = pd.read_csv(input_file_path)


In [4]:
mpra_df = pd.read_csv(MPRA_DATA)
mpra_df

,oligo,RNA_DNA_ratio_log2_ancestral,RNA_DNA_ratio_log2_derived,DNA_counts_raw_ancestral,DNA_counts_raw_derived,RNA_counts_raw_ancestral,RNA_counts_raw_derived,barcode_count_ancestral,barcode_count_derived,alpha_activity_estimate_ancestral,...,normalized_activity_estimate_derived,activity_fdr_ancestral,activity_fdr_derived,activity_ancestral,activity_derived,logFC_derived_vs_ancestral,differential_activity_fdr,differential_activity,sequence_ancestral,sequence_derived
0,seq_100000_chr8:30170404-30170673_SCREEN_a3_L1,0.305667,0.572193,870.0,933.0,3400.0,4386.0,75.0,98.0,2.912881,...,6.972534,0.000003,4.327965e-11,active,active,0.062453,0.634114,False,gtctttatcagcagcatgaaaatgaactaatatacctcctttcttt...,gtctttatcagcagcatgaaaatgaactaatatacctcctttcttt...
1,seq_100001_chr11:26270190-26270459_SCREEN_a3_L1,-0.847603,-0.810681,1120.0,1105.0,1967.0,1991.0,90.0,81.0,1.436428,...,-0.465257,0.852420,8.280452e-01,non_active,non_active,NaN,NaN,NaN,tttattcagacacaatgattctgctgaatgttcaaagattttcttt...,tttattcagacacaatgattctgctgaatgttcaaagattttcttt...
2,seq_100002_chr19:42985016-42985285_SCREEN_a3_L1,0.362402,0.122223,211.0,446.0,860.0,1536.0,33.0,55.0,2.768096,...,2.245034,0.000046,7.787675e-02,active,non_active,-0.271145,0.063988,False,ccctgctatataaactccccagtttcagtctgtcagagattgatct...,ccctgctatataaactccccagtttcagtctgtcagagattgatct...
3,seq_100003_chr8:34830463-34830732_SCREEN_a3_L1,-0.714446,-0.874729,307.0,764.0,592.0,1317.0,29.0,51.0,1.629875,...,-0.777659,0.705492,8.804929e-01,non_active,non_active,NaN,NaN,NaN,agtacaacagaaaagcaaatcatgacgagaatgaagcctgaactct...,agtacaacagaaaagcaaatcatgacgagaatgaagcctgaactct...
4,seq_100004_chr8:17086465-17086734_SCREEN_a3_L1,-1.092105,-0.612584,358.0,271.0,531.0,561.0,14.0,13.0,1.296493,...,0.890924,0.932396,4.841857e-01,non_active,non_active,NaN,NaN,NaN,tttactttgtgaagtatatccttcggaataaatgcattcctggaaa...,tttactttgtgaagtatatccttcggaataaatgcattcctggaaa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355886,seq_99995_chr1:232513928-232514197_SCREEN_a3_L1,-0.842769,-0.549248,326.0,364.0,575.0,787.0,11.0,20.0,1.572049,...,1.192519,0.755333,3.763739e-01,non_active,non_active,NaN,NaN,NaN,agagtgagaccctgtctcaaaaaaaaaaaaaaaatagtttaagtaa...,agagtgagaccctgtctcaaaaaaaaaaaaaaaatagtttaagtaa...
355887,seq_99997_chr8:98081035-98081304_SCREEN_a3_L1,-0.469455,-0.323055,1767.0,1162.0,4033.0,2936.0,120.0,88.0,1.817734,...,1.816562,0.494230,1.707516e-01,non_active,non_active,NaN,NaN,NaN,ggttaaatgacttgcccaagactatactgcccttagggcaggagtc...,ggttaaatgacttgcccaagactatactgcccttagggcaggagtc...
355888,seq_99998_chr3:135890202-135890471_SCREEN_a3_L1,-0.598124,-0.731697,1298.0,1762.0,2710.0,3353.0,89.0,105.0,1.692378,...,-0.077829,0.644568,7.539551e-01,non_active,non_active,NaN,NaN,NaN,aaagctgaatgactccttagggttgtcctatgctgggcccaaggag...,aaagctgaatgactccttagggttgtcctatgctgggcccaaggag...
355889,seq_99999_chr4:182061380-182061649_SCREEN_a3_L1,-0.759305,-0.713852,695.0,1163.0,1298.0,2241.0,55.0,77.0,1.539958,...,-0.411293,0.780191,8.186231e-01,non_active,non_active,NaN,NaN,NaN,ttcattcacacattttaattctttattctcagactgtgcaatcatt...,ttcattcacacattttaattctttattctcagactgtgcaatcatt...


In [8]:
TF_list = [tf for tf in os.listdir(DATA_TF_PATH) if os.path.isdir(os.path.join(DATA_TF_PATH, tf))]
TF_list

['Elf2_mus_musculus',
 'FOXC1',
 'Setbp1_mus_musculus',
 'FoxJ3',
 'Tbx2_mus_musculus',
 'FOXO3a',
 'Bhlhb2_mus_musculus',
 'Arid3a_mus_musculus',
 'Max',
 'NKX2-8',
 'Gata4_mus_musculus',
 'Etv6_mus_musculus',
 'ZNF655',
 'Ar_mus_musculus',
 'MTF2',
 'Foxa3_mus_musculus',
 'En1_mus_musculus',
 'KLF11',
 'VSX1',
 'BCL11B',
 'Ahctf1_mus_musculus',
 'Elf5_mus_musculus',
 'VENTX',
 'Elk1_mus_musculus',
 'POU4F3',
 'Arp1',
 'Dbp_mus_musculus',
 'VSX2',
 'SNAI2',
 'Evx1_mus_musculus',
 'Duxl_mus_musculus',
 'Barx2_mus_musculus',
 'ATF1',
 'Etv3_mus_musculus',
 'Elk3_mus_musculus',
 'TBP',
 'Erg_mus_musculus',
 'FoxN2',
 'NR1H4',
 'Cramp1l_mus_musculus',
 'Bsx_mus_musculus',
 'GST-CSL-NOTCH1',
 'PAX4',
 'KLF1',
 'Evx2_mus_musculus',
 'HESX1',
 'POU6F2',
 'CREB1',
 'POU3F4',
 'FOXR1',
 'PHF1',
 'MSX2',
 'BCL11A',
 'Dobox4_mus_musculus',
 'EGR2',
 'PITX2',
 'HOXB7',
 'Dlx4_mus_musculus',
 'Ets1_mus_musculus',
 'PAX6',
 'Atf3_mus_musculus',
 'GFI1B',
 'Rarg_mus_musculus',
 'PAX7',
 'Dlx1_mus_mu

In [9]:
len(TF_list)

127

Let's try to parse 1 TF data file 

In [11]:
# This is how the TF data looks like. There are 127 such tables. 

EXAMPLE_DIRECTORY = "/home/labs/davidgo/ortalh/TF_analysis_data/data_TF_for_analysis/Ahctf1_mus_musculus"
EXAMPLE_FILE = "Ahctf1_8mers_11111111.txt"
example_text  = pd.read_csv(os.path.join(EXAMPLE_DIRECTORY, EXAMPLE_FILE), sep="\t", header=None)
example_text

,0,1,2,3,4
0,8-mer,8-mer,E-score,Median,Z-score
1,AAAAAAAA,TTTTTTTT,0.48566,3637.47,9.0057
2,AAAAAAAC,GTTTTTTT,0.44921,2113.72,7.2302
3,AAAAAAAG,CTTTTTTT,0.44895,1536.79,6.1877
4,AAAAAAAT,ATTTTTTT,0.48716,3468.56,8.8501
...,...,...,...,...,...
32892,TTTCCAAA,TTTGGAAA,0.32919,569.51,2.9411
32893,TTTCGAAA,TTTCGAAA,0.24944,347.95,1.3296
32894,TTTGAAAA,TTTTCAAA,0.34429,677.94,3.5111
32895,TTTGCAAA,TTTGCAAA,0.24837,374.90,1.5736


Seems easy, now I will read all TFs 

In [37]:
### This is ortal's code to extract the TF data from the files
print("##### Extracting txt files from TF path #####")
print("##### Reading TF files #####")
TF_data_files = ortals.filter_TF_txt_files(ortals.get_all_txt_in_folder(DATA_TF_PATH))

# TF_dict_all = {}
# count_all_TF = 0
# TF_dict_all_opp = {}
# mean_intensity_for_TF = {}
# TF_list_filtered_out = []
# k_list = []
# for curr_TF_path in list_TF_path_filtered:
#     curr_TF_data = open_txt(curr_TF_path)
#     TF_name = curr_TF_path.split(SLASH)[7]
#     result = creating_TF_list_new(curr_TF_data, TF_name, TF_dict_all, TF_dict_all_opp, k_list)
#     if result:
#         TF_dict_all, TF_dict_all_opp, k_list, mean_intensity = result[0], result[1], result[2], result[3]
#         mean_intensity_for_TF[TF_name] = mean_intensity
#         count_all_TF += 1
#     else:
#         print("####### filtered out! file not in the right format! : {}".format(curr_TF_path))
#         TF_list_filtered_out.append(curr_TF_path)
#     print("####### overall not in the right format: {}".format(len(TF_list_filtered_out)))
#     print("####### overall in the right format: {}".format(count_all_TF))

##### Extracting txt files from TF path #####
##### Reading TF files #####


In [ ]:
file0 = pd.read_csv(TF_data_files[4], sep="\t", header=None)
file1 = pd.read_csv(TF_data_files[1], sep="\t", header=None) 
file0

,0,1,2,3,4
0,8-mer,8-mer,E-score,Median,Z-score
1,AAAAAAAA,TTTTTTTT,0.09474,468.40,-0.2041
2,AAAAAAAC,GTTTTTTT,0.15267,598.20,0.7683
3,AAAAAAAG,CTTTTTTT,0.08452,607.57,0.8301
4,AAAAAAAT,ATTTTTTT,0.15318,664.55,1.1866
...,...,...,...,...,...
32892,TTTCCAAA,TTTGGAAA,0.08026,635.62,1.0096
32893,TTTCGAAA,TTTCGAAA,-0.04401,598.35,0.7694
32894,TTTGAAAA,TTTTCAAA,0.12137,759.41,1.7171
32895,TTTGCAAA,TTTGCAAA,0.23192,796.68,1.9076


In [88]:
def parse_8mer_tf_file(file_name):
    """
    Parse the TF file and return a dataframe with 5 columns.
    """
    df = pd.read_csv(file_name, sep="\t", header=None)
    if "AAA" not in df.iloc[0, 0]:
        df = df.iloc[1:].reset_index(drop=True)
    if "A" * 8 != df.iloc[0, 0]:
        return None
    df = df.iloc[:, :5]
    TF_name = file_name.split(SLASH)[7]
    df.columns = ["8-mer", "reverse_complement", f"E-score_{TF_name}", f"Median_{TF_name}", f"Z-score_{TF_name}"]
    
    return df

df1 = parse_8mer_tf_file(TF_data_files[1])
df2 = parse_8mer_tf_file(TF_data_files[4])


# df1.columns = COLUMN_NAMES
# df2.columns = COLUMN_NAMES


In [79]:
df_merged = df1.merge(df2, on=["8-mer", "reverse_complement"], how="inner", suffixes=("_1", "_2"))
print(df_merged.head())
df_merged.columns = pd.MultiIndex.from_tuples(
    [("ID", "8-mer"), ("ID", "reverse_complement"),
     ("df1", "E-score"), ("df1", "Median"), ("df1", "Z-score"),
     ("df2", "E-score"), ("df2", "Median"), ("df2", "Z-score")]
)
df_merged.head()



      8-mer reverse_complement E-score_FOXC1 Median_FOXC1 Z-score_FOXC1  \
0  AAAAAAAA           TTTTTTTT       0.27761     67581.95        1.7294   
1  AAAAAAAC           GTTTTTTT       0.13557     66630.52        1.3546   
2  AAAAAAAG           CTTTTTTT       0.24169     67778.93        1.8064   
3  AAAAAAAT           ATTTTTTT       0.12254     65436.85        0.8766   
4  AAAAAACA           TGTTTTTT       0.27872     68385.35        2.0418   

  E-score_Tbx2_mus_musculus Median_Tbx2_mus_musculus Z-score_Tbx2_mus_musculus  
0                   0.09474                   468.40                   -0.2041  
1                   0.15267                   598.20                    0.7683  
2                   0.08452                   607.57                    0.8301  
3                   0.15318                   664.55                    1.1866  
4                   0.17689                   662.94                    1.1769  


ID                         df1                        df2          \
      8-mer reverse_complement  E-score    Median Z-score  E-score  Median   
0  AAAAAAAA           TTTTTTTT  0.27761  67581.95  1.7294  0.09474  468.40   
1  AAAAAAAC           GTTTTTTT  0.13557  66630.52  1.3546  0.15267  598.20   
2  AAAAAAAG           CTTTTTTT  0.24169  67778.93  1.8064  0.08452  607.57   
3  AAAAAAAT           ATTTTTTT  0.12254  65436.85  0.8766  0.15318  664.55   
4  AAAAAACA           TGTTTTTT  0.27872  68385.35  2.0418  0.17689  662.94   

            
   Z-score  
0  -0.2041  
1   0.7683  
2   0.8301  
3   1.1866  
4   1.1769

In [148]:
multi_tf_df = parse_8mer_tf_file(TF_data_files[0])
for i in range(1, len(TF_data_files)):
# for i in range(1, 3):
    df = parse_8mer_tf_file(TF_data_files[i])
    if df is None:
        continue
    multi_tf_df = multi_tf_df.merge(df, on=["8-mer", "reverse_complement"], how="inner", suffixes=("", ""))
    print(i, multi_tf_df.shape)


1 (32896, 8)
2 (32896, 11)
3 (32896, 14)
4 (32896, 17)
5 (32896, 20)
6 (32896, 23)
7 (32896, 26)
8 (32896, 29)
9 (32896, 32)
10 (32896, 35)
11 (32896, 38)
12 (32896, 41)
13 (32896, 44)
14 (32896, 47)
15 (32896, 50)
16 (32896, 53)
17 (32896, 56)
18 (32896, 59)
19 (32896, 62)
20 (32896, 65)
21 (32896, 68)
22 (32896, 71)
23 (32896, 74)
24 (32896, 77)
26 (32896, 80)
27 (32896, 83)
28 (32896, 86)
29 (32896, 89)
30 (32896, 92)
31 (32896, 95)
33 (32896, 98)
34 (32896, 101)
36 (32896, 104)
37 (32896, 107)
38 (32896, 110)
39 (32896, 113)
40 (32896, 116)
41 (32896, 119)
42 (32896, 122)
43 (32896, 125)
44 (32896, 128)
45 (32896, 131)
46 (32896, 134)
48 (32896, 137)
49 (32896, 140)
50 (32896, 143)
51 (32896, 146)
52 (32896, 149)
53 (32896, 152)
54 (32896, 155)
55 (32896, 158)
56 (32896, 161)
57 (32896, 164)
58 (32896, 167)
59 (32896, 170)
60 (32896, 173)
61 (32896, 176)
62 (32896, 179)
63 (32896, 182)
64 (32896, 185)
65 (32896, 188)
66 (32896, 191)
67 (32896, 194)
68 (32896, 197)
69 (32896, 200)
7

In [93]:
multi_tf_df

,8-mer,reverse_complement,E-score_Elf2_mus_musculus,Median_Elf2_mus_musculus,Z-score_Elf2_mus_musculus,E-score_FOXC1,Median_FOXC1,Z-score_FOXC1,E-score_Setbp1_mus_musculus,Median_Setbp1_mus_musculus,...,Z-score_Eomes_mus_musculus,E-score_OVOL2,Median_OVOL2,Z-score_OVOL2,E-score_HOXD13,Median_HOXD13,Z-score_HOXD13,E-score_SIX6,Median_SIX6,Z-score_SIX6
0,AAAAAAAA,TTTTTTTT,0.32485,4461.57,1.8518,0.27761,67581.95,1.7294,0.40331,936.26,...,1.117451,0.31938,95222.60,2.9889,0.22284,94047.88,0.9885,-0.01591,5630.91,0.4855
1,AAAAAAAC,GTTTTTTT,0.08518,3822.72,0.4609,0.13557,66630.52,1.3546,0.32338,834.76,...,0.820674,0.26739,83166.54,2.1622,0.22298,96486.23,1.2080,-0.28454,4774.14,-0.2865
2,AAAAAAAG,CTTTTTTT,0.19728,4298.43,1.5165,0.24169,67778.93,1.8064,0.38597,813.63,...,1.475557,0.18495,70497.27,1.1529,0.19466,91025.98,0.7086,-0.01487,5392.32,0.2830
3,AAAAAAAT,ATTTTTTT,0.24812,4048.72,0.9779,0.12254,65436.85,0.8766,0.45527,1274.52,...,0.441932,0.30132,86229.21,2.3830,0.27454,99240.27,1.4492,-0.08425,5195.54,0.1091
4,AAAAAACA,TGTTTTTT,0.17321,4142.24,1.1834,0.27872,68385.35,2.0418,0.29179,714.27,...,2.604086,0.29346,83422.52,2.1809,0.20770,96814.59,1.2371,-0.17977,4647.98,-0.4118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32891,TTTCCAAA,TTTGGAAA,0.19685,4743.74,2.4037,0.19882,65749.16,1.0025,0.19619,535.40,...,-0.359307,0.15343,73947.14,1.4447,0.07722,90375.48,0.6471,-0.12015,5327.45,0.2264
32892,TTTCGAAA,TTTCGAAA,0.18924,4441.43,1.8110,0.07948,63723.77,0.1753,0.31579,751.97,...,0.756326,0.16588,76482.24,1.6505,0.11076,94457.49,1.0258,-0.21152,4579.73,-0.4810
32893,TTTGAAAA,TTTTCAAA,0.20735,4469.74,1.8682,0.23589,68938.19,2.2547,0.36011,851.56,...,0.382383,0.32045,97027.20,3.1035,0.22801,99414.57,1.4642,-0.19790,4253.60,-0.8265
32894,TTTGCAAA,TTTGCAAA,-0.06517,3744.02,0.2737,-0.03811,63378.00,0.0314,0.26053,692.10,...,2.718907,0.08961,68022.82,0.9347,0.03050,92387.23,0.8358,0.04524,5198.53,0.1118


In [ ]:
parts = 'E-score_Elf2_mus_musculus'.split("_", 1)
parts

['E-score', 'Elf2_mus_musculus']

In [98]:
def convert_into_multiindex(df):
    """
    Convert the dataframe into a multiindex dataframe.
    """
    id_cols = ["8-mer", "reverse_complement"]
    multi_columns = []
    for col in df.columns:
        if col in id_cols:
            multi_columns.append(("ID", col))
        else:
            parts = col.split("_", 1)
            if len(parts) == 2:
                category, tf_name = parts
            else:
                category, tf_name = col, "unkown"
            multi_columns.append((category, tf_name))
    df.columns = pd.MultiIndex.from_tuples(multi_columns)
    return df



multi_tf_df_converted = convert_into_multiindex(multi_tf_df)
import ace_tools as tools
tools.display_dataframe_to_user(name="MultiIndexed DataFrame", dataframe=multi_tf_df_converted)

ModuleNotFoundError: No module named 'ace_tools'

In [113]:
multi_tf_df_converted.sort_index(axis=1,level=1, inplace=True)
multi_tf_df_converted

,ID,E-score,Median,Z-score,E-score,Median,Z-score,E-score,Median,Z-score,...,E-score,Median,Z-score,E-score,Median,Z-score,E-score,Median,Z-score,ID
,8-mer,ARX,ARX,ARX,Ahctf1_mus_musculus,Ahctf1_mus_musculus,Ahctf1_mus_musculus,Alx3_mus_musculus,Alx3_mus_musculus,Alx3_mus_musculus,...,WT1,WT1,WT1,ZNF200,ZNF200,ZNF200,ZNF655,ZNF655,ZNF655,reverse_complement
0,AAAAAAAA,-0.26798,21446.63,-2.5642,0.48566,3637.47,9.0057,0.28589,4023.19,2.3401,...,0.20493,19119.68,1.3045,0.37638,75351.10,1.6922,0.07872,15953.20,0.1415,TTTTTTTT
1,AAAAAAAC,-0.37748,22328.68,-2.1900,0.44921,2113.72,7.2302,0.13567,3359.73,1.3416,...,0.14171,17718.71,0.8530,0.31089,75622.63,1.7340,-0.01151,15689.73,0.0316,GTTTTTTT
2,AAAAAAAG,-0.34844,20495.03,-2.9855,0.44895,1536.79,6.1877,0.18368,3357.92,1.3386,...,0.24876,19536.28,1.4324,0.28287,76116.79,1.8098,0.03539,16858.92,0.5059,CTTTTTTT
3,AAAAAAAT,-0.18697,23135.28,-1.8606,0.48716,3468.56,8.8501,0.29545,3875.86,2.1334,...,0.23145,18660.37,1.1602,0.29767,73604.92,1.4194,0.18357,18159.88,0.9966,ATTTTTTT
4,AAAAAACA,-0.26721,22133.28,-2.2716,0.42078,1196.18,5.3682,0.17853,3580.97,1.6949,...,0.20801,19797.97,1.5113,0.28388,75508.02,1.7164,0.01495,16335.11,0.2976,TGTTTTTT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32891,TTTCCAAA,-0.15100,27545.35,-0.2410,0.32919,569.51,2.9411,0.08317,2916.69,0.5581,...,0.14190,17472.34,0.7699,0.04465,70102.26,0.8522,0.06913,17261.48,0.6617,TTTGGAAA
32892,TTTCGAAA,-0.26713,21956.48,-2.3461,0.24944,347.95,1.3296,0.01566,3057.97,0.8202,...,0.14762,17541.30,0.7933,0.01721,65444.80,0.0525,0.00759,16656.96,0.4264,TTTCGAAA
32893,TTTGAAAA,-0.24814,22929.79,-1.9434,0.34429,677.94,3.5111,0.27189,3533.30,1.6207,...,0.27712,18814.81,1.2091,0.21600,74509.58,1.5615,0.12263,17199.88,0.6381,TTTTCAAA


In [146]:
multi_tf_df_converted.loc[:, (slice(None), ["8-mer","Elf2_mus_musculus"])]

,ID,E-score,Median,Z-score
,8-mer,Elf2_mus_musculus,Elf2_mus_musculus,Elf2_mus_musculus
0,AAAAAAAA,0.32485,4461.57,1.8518
1,AAAAAAAC,0.08518,3822.72,0.4609
2,AAAAAAAG,0.19728,4298.43,1.5165
3,AAAAAAAT,0.24812,4048.72,0.9779
4,AAAAAACA,0.17321,4142.24,1.1834
...,...,...,...,...
32891,TTTCCAAA,0.19685,4743.74,2.4037
32892,TTTCGAAA,0.18924,4441.43,1.8110
32893,TTTGAAAA,0.20735,4469.74,1.8682


In [51]:
for i in wierd_files:
    df = pd.read_csv(TF_data_files[i], sep="\t", header=None)
    print("-------------------")
    print(f"File numbe: {TF_data_files[i]}", end="\n\n")
    print(df.head(3), end="\n\n")


-------------------
File numbe: /home/labs/davidgo/ortalh/TF_analysis_data/data_TF_for_analysis/Arp1/ARP_Normalized_7mers_1111111.txt

         0        1        2        3        4
0    7-mer    7-mer  E-score   Median  Z-score
1  AAAAAAA  TTTTTTT  0.21077  8932.11   1.7041
2  AAAAAAC  GTTTTTT  0.19207  8588.51   0.5490

-------------------
File numbe: /home/labs/davidgo/ortalh/TF_analysis_data/data_TF_for_analysis/ATF1/Norm_ATF1__488nm_500_70/ATF1_Normalisation_7mers_1111111.txt

         0        1         2        3        4
0    7-mer    7-mer   E-score   Median  Z-score
1  AAAAAAA  TTTTTTT  -0.18936   861.03  -2.6481
2  AAAAAAC  GTTTTTT  -0.00044  1047.59  -1.4261

-------------------
File numbe: /home/labs/davidgo/ortalh/TF_analysis_data/data_TF_for_analysis/TBP/results_1-8_TBP_10nM/TBP_10nM_ch1-8_7mers_1111111.txt

         0        1        2       3        4
0    7-mer    7-mer  E-score  Median  Z-score
1  AAAAAAA  TTTTTTT  0.32639  203.36   2.9670
2  AAAAAAC  GTTTTTT  0.3062